In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline



In [2]:
data=pd.read_csv("EdStatsData.csv")
countries=pd.read_csv("EdStatsCountry.csv")
indicators=pd.read_csv("EdStatsSeries.csv")
infoindicators=pd.read_csv("EdStatsFootNote.csv")
countryseries=pd.read_csv("EdStatsCountry-Series.csv")

FileNotFoundError: [Errno 2] File EdStatsData.csv does not exist: 'EdStatsData.csv'

In [ ]:
data.dropna(axis=1,how='all',inplace=True)
countries.dropna(axis=1,how='all',inplace=True)
indicators.dropna(axis=1,how='all',inplace=True)
infoindicators.dropna(axis=1,how='all',inplace=True)
countryseries.dropna(axis=1,how='all',inplace=True)

In [ ]:
print(data.shape)
print(countries.shape)
print(indicators.shape)
print(infoindicators.shape)
print(countryseries.shape)

In [ ]:
data.head(2)

In [ ]:
countryseries.dropna(axis=1,how='all',inplace=True) #J'élimine la colonne avec tous les NaN

In [ ]:
countryseries.isnull().values.any()

In [ ]:
countryseries.drop_duplicates()

Le tablea countryseries est propre. On a supprimé une colonne et il reste donc 3 colonne et 612 lignes. Ce tableau représente des codes (d'études) asosciés à des codes pays et les sources d'information.

In [ ]:
infoindicators.head(2)

In [ ]:
infoindicators.dropna(axis=1,how='all',inplace=True)

In [ ]:
infoindicators.isnull().values.any()

In [ ]:
def changedate(date):
    return int(date[2:])

infoindicators['Year']=infoindicators['Year'].apply(changedate)
infoindicators.shape

Au final on a un tableau avec 643638 lignes et 4 colonnes.
Ce tableau semble nous donner encore une liste d'études par code de pays et par année.
On regarde si il n'y aurait pas de doublons:

In [ ]:
infoindicators.drop_duplicates()
infoindicators.shape

In [ ]:
infoindicators.head(3)

In [ ]:
countries.dropna(axis=1,how='all',inplace=True)
countries.shape

On a cette fois 241 lignes et 31 colonnes.

In [ ]:
countries.columns

In [ ]:
#On peut déjà supprimer quelques colonnes qui ne nous servent pas:
#Juste afin de pouvoir observer toutes les lignes

L=['National accounts base year','National accounts reference year','System of National Accounts','Currency Unit','External debt Reporting status']
for i in L:
    del countries[i]

In [ ]:
countries.head()

In [ ]:
#On peut encore enlever des colonnes
L=['WB-2 code', 'Government Accounting concept','IMF data dissemination standard', 'Vital registration complete','Latest agricultural census','Latest water withdrawal data']
for i in L:
    del countries[i]

In [ ]:
countries.head()

In [ ]:
L=['2-alpha code','SNA price valuation','Lending category','Other groups','Alternative conversion factor','PPP survey year','Balance of Payments Manual in use']
for i in L:
    del countries[i]

In [ ]:
countries[4:6]

In [ ]:
countries.shape

In [ ]:
countries[countries.duplicated(['Country Code'])]

Il ne semble pas y avoir de doublon


In [ ]:
# On va créer un nouveau tableau avec juste les régions
regions=countries[countries.Region.isnull()]
countries.dropna(subset=['Region'],how='all',inplace=True)
print(regions.shape,countries.shape)

In [ ]:
indicators.shape

In [ ]:
indicators.dropna(axis=1,how='all',inplace=True)


In [ ]:
indicators[indicators.duplicated(['Series Code'])]

In [ ]:
indicators.dropna(subset=['Series Code'],how='any',inplace=True)
indicators.shape

In [ ]:
indicators.head()

Après nettoyage on a donc 3665 lignes et 15 colonnes.
Cette table nous donne plus d'information sur les codes des études que l'on avait trouvé dans countryseries et indicatornotes

In [ ]:
data.shape

data a 886930 lignes et 70 colonnes

In [ ]:
#On supprimes les colonnes avec que des nan:
data.dropna(axis=1, how='all', inplace=True)
data.shape

In [ ]:
data.head()

Ce tableau semble montrer les résultats des différentes études par pays et par code d'indicateur entre 1970 et 2100

In [ ]:
#On vérifie les colonnes
data.columns

In [ ]:
#On supprime éventuellement les indicateurs qui n'apportent aucune information (NaN entre 1970 et 2100)
a=list(data.columns)[4:]
data.dropna(subset=a,how='all',inplace=True)
data.shape

In [ ]:
#On regarde si il n'y a pas des doublons:
data[data.duplicated(['Country Code','Indicator Code'])]

Il ne semble donc pas y avoir de doublons

Pour les premiers indicateurs on va rechercher dans la table des indicateurs.
On voit sur le site de la WB qu'il y a un indicateur pour le nombre d'ordinateur pour 100 habitants et l'acces à internet pour 100 habitants.
On va essayer de croiser ces indicateurs avec les population de 15-24 qui sont la cible de l'entreprise (secondaire et université).


In [ ]:
indicators.Topic.unique()

In [ ]:
indicators[indicators.Topic=='Infrastructure: Communications']

On va donc commencer par les indicateurs: IT.CMP.PCMP.P2 et IT.NET.USER.P2
Et on va chercher les populations de 15-24


In [ ]:
a=[]
for i in indicators['Indicator Name']:
    if '15-24' in i.lower():
        a.append(i)
print(len(a),a)

In [ ]:
indicators[indicators['Indicator Name']=='Population, ages 15-24, total']

In [ ]:
indicpop=[]
for a in indicators['Series Code']:
    if 'SP.POP' in a and 'FE' not in a and 'MA' not in a:
        indicpop.append(a)
print(len(indicpop),indicpop)

Voyons si ces informations sont disponibles pour une grande partie des pays:

In [ ]:
data[data['Indicator Code']=='SP.POP.1524.TO.UN'].shape

In [ ]:
data[data['Indicator Code']=='SP.POP.TOTL'].shape

On va créer 2 sous tables une pour les ordinateurs et une pour internet avec les population de 15-24 et les populations totales qu'on va commencer à étudier afin de faire une première préselection sur le volume de clients potentiels dans le pays.
On verra ensuite comment affiner avec d'autres indicateurs supplémentaires type:
Niveaux de revenus dans le pays, niveau des enseignants, projections des chiffres sur les prochaines années...


In [ ]:
internet=['SP.POP.1524.TO.UN','SP.POP.TOTL','IT.NET.USER.P2']
population_internet=pd.DataFrame()

for i in internet:
    population_internet=pd.concat([population_internet,data[data['Indicator Code']==i]],ignore_index=True)

In [ ]:
computer=['SP.POP.1524.TO.UN','SP.POP.TOTL','IT.CMP.PCMP.P2']
population_computer=pd.DataFrame()

for i in computer:
    population_computer=pd.concat([population_computer,data[data['Indicator Code']==i]],ignore_index=True)

In [ ]:
population_internet.columns

Globalement dans le monde, la population des pays croit ou diminue vraiment très légérement, de même le nombre d'ordinateur par habitant et l'accés à internet est en croissance constante dans les pays.
Pour tous ces indicateurs on va donc prendre les valeurs maximales dans les pays.

In [ ]:
population_internet['Max']=population_internet.max(axis=1,numeric_only=True)
echantillon_internet=population_internet[['Country Name','Country Code','Indicator Name','Max']].copy()
echantillon_internet.columns=['Country','Code','Indicator','Value']
def modifinternet(x):
    if '15-24' in x:
        return '15-24'
    elif 'Population, total' in x:
        return 'total'
    else:
        return 'internet'
echantillon_internet.Indicator=echantillon_internet.Indicator.apply(modifinternet)
echantillon_internet.sort_values(by=['Code','Indicator'],inplace=True)

In [ ]:
population_computer['Max']=population_computer.max(axis=1,numeric_only=True)
echantillon_computer=population_computer[['Country Name','Country Code','Indicator Name','Max']].copy()
echantillon_computer.columns=['Country','Code','Indicator','Value']


def modifcomputer(x):
    if '15-24' in x:
        return '15-24'
    elif 'Population, total' in x:
        return 'total'
    else:
        return 'computer'
echantillon_computer.Indicator=echantillon_computer.Indicator.apply(modifcomputer)
echantillon_computer.sort_values(by=['Code','Indicator'],inplace=True)

In [ ]:
echantillon_internet.head()

In [ ]:

internet=pd.DataFrame(columns=['Country','Internet','Population','15-24'])

for i in range(countries.shape[0]):
    code=countries.iloc[i]['Country Code']
    df=echantillon_internet[echantillon_internet.Code==code]
    if df[df.Indicator=='internet'].empty or df[df.Indicator=='15-24'].empty and df[df.Indicator=='total'].empty:
        continue
    else:
        D={'Country':df[df.Indicator=='internet'].iloc[0]['Country'],\
           'Internet':df[df.Indicator=='internet'].iloc[0]['Value']}
        if not df[df.Indicator=='total'].empty:
            D['Population']=df[df.Indicator=='total'].iloc[0]['Value']
        if not df[df.Indicator=='15-24'].empty:
            D['15-24']=df[df.Indicator=='15-24'].iloc[0]['Value']
    for cle,valeur in D.items():
        internet.loc[code,cle]=valeur

computer=pd.DataFrame(columns=['Country','Computer','Population','15-24'])

for i in range(countries.shape[0]):
    code=countries.iloc[i]['Country Code']
    df=echantillon_computer[echantillon_computer.Code==code]
    if df[df.Indicator=='computer'].empty or df[df.Indicator=='15-24'].empty and df[df.Indicator=='total'].empty:
        continue
    else:
        D={'Country':df[df.Indicator=='computer'].iloc[0]['Country'],'Computer':df[df.Indicator=='computer'].iloc[0]['Value']}
        if not df[df.Indicator=='total'].empty:
            D['Population']=df[df.Indicator=='total'].iloc[0]['Value']
        if not df[df.Indicator=='15-24'].empty:
            D['15-24']=df[df.Indicator=='15-24'].iloc[0]['Value']
    for cle,valeur in D.items():
        computer.loc[code,cle]=valeur

In [ ]:
internet.head()

In [ ]:
computer.head()

In [ ]:
internet.isnull().sum()

In [ ]:
computer.isnull().sum()

In [ ]:
#je vais remplir les 15-24 vides en utilisant la moyenne de population de 15-24 sur les pays
df=internet.dropna(how='all')
a=df['15-24']/df['Population']
moyenneint=a.mean()

for (i,country) in internet.iterrows():
    if pd.isnull(country['15-24']):
        internet.loc[i,'15-24']=internet.loc[i,'Population']*moyenneint
        
df=computer.dropna(how='all')
a=df['15-24']/df['Population']
moyennecmp=a.mean()

for (i,country) in computer.iterrows():
    if pd.isnull(country['15-24']):
        computer.loc[i,'15-24']=computer.loc[i,'Population']*moyennecmp


In [ ]:
internet.head()
    

In [ ]:
computer.head()

In [ ]:
internet['TargetPop']=internet['Internet']*internet['15-24']/100
print('min:',internet['TargetPop'].min(),'max:',internet['TargetPop'].max(),\
      'moyenne:',internet['TargetPop'].mean(),'sigma:',internet['TargetPop'].std(),'mediane:',internet['TargetPop'].median())


In [ ]:
computer['TargetPop']=computer['Computer']*computer['15-24']/100
print(computer['TargetPop'].min(),computer['TargetPop'].max(),computer['TargetPop'].mean())


In [ ]:
computer.head()

In [ ]:
techno=pd.merge(internet,computer,on=['Country'],how='outer',suffixes=['_internet','_computer'])
techno=techno[['Country','TargetPop_internet','TargetPop_computer']]
techno.fillna(0,inplace=True)
techno.head()

fig = plt.figure(figsize=(9,6))
ax = plt.axes()
x=techno['TargetPop_internet']
y=techno['TargetPop_computer']
plt.scatter(x, y, c = 'blue')

plt.title("Accès ordinateur en fonction de accés internet",size=20)


# Titres des axes
ax = ax.set(xlabel='Accès internet', ylabel='Accès ordinateur')


In [ ]:
techno.head()

In [ ]:

techno.columns=['Country','int','comp']
groupes = ['int','comp'] # va recevoir les données agrégées à afficher

fig = plt.figure(figsize=(5,7))

a=plt.boxplot([techno[g] for g in groupes],
            positions= [15,20], # abscisses des boxplots
            showfliers= True, # on ne prend pas en compte les outliers
            widths= 1, # largeur graphique des boxplots
)
plt.gca().xaxis.set_ticklabels(['internet', 'computer'],size=15)
for element in a['medians']:
    element.set_color('blue')
for element in a['caps']:
    element.set_color('red')

plt.xlim(12,22)
plt.ylim(0,4000000)

plt.title('Diagrammes en boite',size=18)

plt.show()



In [ ]:
fig = plt.figure(figsize=(6,6))
ax = plt.axes()

plt.hist([internet['TargetPop']],range=(-1000000,139000000),bins=56)
ax = ax.set(xlabel='Personnes', ylabel='Nombre de Pays')

plt.title('Nombre de personnes avec accés à internet')
plt.show()

In [ ]:
# On va déja éliminer tous les pays avec moins de 1 500 000 de 15-24 avec un accés internet

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = plt.axes()

plt.hist([computer['TargetPop']],range=(-1000000,139000000),bins=56)
ax = ax.set(xlabel='Personnes', ylabel='Nombre de Pays')

plt.title('Nombre de personnes avec accés à un ordinateur')
plt.show()

In [ ]:
#On va également supprimer les pays avec moins de 1 000 000 15-24 ans avec un accés à un ordinateur

In [ ]:
techno.head()

In [ ]:
totinternet=techno.int.sum()

In [ ]:
totcomp=techno.comp.sum()
print(totinternet,totcomp)

In [ ]:
payscibles=pd.merge(internet[internet.TargetPop>1500000],computer[computer.TargetPop > 1000000],on=['Country','Population'],how='outer',suffixes=['_internet','_computer'])
payscibles.dropna(how='any',inplace=True)
payscibles=pd.merge(payscibles,countries[['Country Code','Table Name']],left_on=['Country'],right_on=['Table Name'])
payscibles=payscibles[['Country Code','Country','Population','TargetPop_internet','TargetPop_computer']]
payscibles.head()

In [ ]:
#On va vérifier qu'on a pas enlevé trop de "population cible":
piepays=payscibles.copy()
piepays.loc['Reste du Monde']=['XXX','Other countries','xxx',totinternet-payscibles['TargetPop_internet'].sum(),totcomp-payscibles['TargetPop_computer'].sum()]
piepays

In [ ]:
import matplotlib as mpl

N=24
fig=plt.figure(figsize = (20, 40))
x = piepays['TargetPop_internet'].tolist()
X=list(np.zeros(23))
X.append(0.2)

cmap=plt.cm.terrain
cmaplist = [cmap(i) for i in range(256,1,-10)] 

fig.add_subplot(1,2,1)
plt.pie(x, labels = piepays.Country.tolist(),colors=cmaplist, \
           explode = X,pctdistance = 0.7, labeldistance = 1.1)
plt.title('15-24 with internet access',size=15)

x = piepays['TargetPop_computer'].tolist()
fig.add_subplot(1,2,2)
plt.pie(x, labels = piepays.Country.tolist(),colors=cmaplist, \
           explode = X,pctdistance = 0.7, labeldistance = 1.1)
plt.title('15-24 with computer access',size=15)


plt.show()


In [ ]:
plt.cm.terrain


In [ ]:
X=np.zeros(23)

In [ ]:
Y=X.tolist().append(0.2)

In [ ]:
codepays=list(payscibles['Country Code'].unique())

In [ ]:
allcountries=data['Country Code'].unique()
dataech=data.copy()
for i in allcountries:
    if i not in codepays:
        dataech=dataech[dataech['Country Code']!=i]

In [ ]:
#Pour affiner ma sélection de pays je vais essayer de trouver des indicateurs disponibles dans la pluspart des pays

In [ ]:
somme=dataech.groupby('Indicator Code').count()
somme.head()

In [ ]:
#Je sélectionne les indicateurs disponibles dans au moins 80% des pays de mon échantillons soit au moins 9
presel=somme[somme['Country Name']>=19]

In [ ]:
L=presel.index.values.tolist()

In [ ]:
indicatorsech=indicators[indicators['Series Code'].isin(L)]

In [ ]:
indicatorsech.shape

In [ ]:
indicatorsech.Topic.unique()

In [ ]:
selection=['Attainment','Learning Outcomes','Teachers','Background','Secondary', 'Tertiary', 'Expenditures']

In [ ]:
indicatorsech2=indicatorsech[indicatorsech.Topic.isin(selection)]

In [ ]:
indicatorsech2.shape

In [ ]:
#On va regarder ces indicateurs un peu plus en détail
indicatorsech[indicatorsech.Topic=='Expenditures']['Indicator Name'].unique()
#ici on va garder:
#'Expenditure on secondary as % of total government expenditure (%)' et 
#'Expenditure on tertiary as % of total government expenditure (%)'
indicatorsech[indicatorsech.Topic=='Tertiary']['Indicator Name'].unique()
#On garde
#'Total inbound internationally mobile students, both sexes (number)'
#'Enrolment in tertiary education per 100,000 inhabitants, both sexes'
indicatorsech[indicatorsech.Topic=='Secondary']['Indicator Name'].unique()
#La on va filtrer un peu plus pour y voir plus clair
indicatorsech[indicatorsech.Topic=='Background']['Indicator Name'].unique()
#On vire
indicatorsech[indicatorsech.Topic=='Teachers']['Indicator Name'].unique()
#On garde:
#'Pupil-teacher ratio in tertiary education (headcount basis)',
#'Pupil-teacher ratio in secondary education (headcount basis)',
indicatorsech[indicatorsech.Topic=='Learning Outcomes']['Indicator Name'].unique()
#Trop spécifique on vire
indicatorsech[indicatorsech.Topic=='Attainment']['Indicator Name'].unique()
#La aussi on va filtrer un peu plus

In [ ]:
#Pour Attainment et Secondary on va commencer par enlever tous les indicateurs avec des termes male, female et primary
echantillon=indicatorsech[indicatorsech.Topic.isin(['Attainment','Secondary'])]
L=[]
for i in indicators['Indicator Name'].unique().tolist():
    if 'male and female' in i.lower() or ('female' not in i.lower() and 'male' not in i.lower() and 'primary' not in i.lower()):
        L.append(i)
echantillon=echantillon[echantillon['Indicator Name'].isin(L)]
echantillon['Indicator Name'].unique()
#On va donc garder
#'Barro-Lee: Percentage of population age 15+ with secondary schooling. Total (Incomplete and Completed Secondary)',
#'Barro-Lee: Percentage of population age 15+ with tertiary schooling. Total (Incomplete and Completed Tertiary)'
#'Enrolment in secondary education, private institutions, both sexes (number)',
#'Projection: Mean years of schooling. Age 20-39. Total',
#'Projection: Population in thousands by highest level of educational attainment. Post Secondary. Total',

On va donc conserver pour l'instant 11 indicateurs

In [ ]:
specific_indicators=['Expenditure on secondary as % of total government expenditure (%)',\
                     'Expenditure on tertiary as % of total government expenditure (%)',\
                     'Total inbound internationally mobile students, both sexes (number)',\
                     'Enrolment in tertiary education per 100,000 inhabitants, both sexes',\
                     'Pupil-teacher ratio in tertiary education (headcount basis)',\
                     'Pupil-teacher ratio in secondary education (headcount basis)',\
                     'Barro-Lee: Percentage of population age 15+ with secondary schooling. Total (Incomplete and Completed Secondary)',\
                     'Barro-Lee: Percentage of population age 15+ with tertiary schooling. Total (Incomplete and Completed Tertiary)',\
                     'Enrolment in secondary education, private institutions, both sexes (number)',\
                     'Projection: Mean years of schooling. Age 20-39. Total',\
                     'Projection: Population in thousands by highest level of educational attainment. Post Secondary. Total']
      

In [ ]:
specific_codes=indicators[indicators['Indicator Name'].isin(specific_indicators)]['Series Code'].unique().tolist()

In [ ]:
echantillon=dataech[dataech['Indicator Code'].isin(specific_codes)]
echantillon.iloc[0]['Indicator Name']

In [ ]:
newdata=payscibles.copy()
fig = plt.figure(figsize=(28,14))
i=0
for code in specific_codes:
    #je fais une copie de la sélection de mon indicateur
    ech=echantillon[echantillon['Indicator Code']==code].copy()
    #je nettoie cette copie en supprimant les colonne dont je n'ai pas besoin:
    indicateur=ech.iloc[0]['Indicator Name']
    if len(indicateur)>60:
        indicateur=indicateur[:60]
    ech.dropna(axis=1,how='all',inplace=True)
    del ech['Country Code']
    del ech['Indicator Name']
    del ech['Indicator Code']
    #Je déplace la colonne country en fin de tableau pour faire un ffill
    ech['Country']=ech['Country Name']
    del ech['Country Name']
    ech=ech.ffill(axis=1)
    ech2=ech[[ech.columns.tolist()[-1],ech.columns.tolist()[-2]]]
    ech2.columns=['Country',indicateur]
    newdata=pd.merge(newdata,ech2,on='Country',how='outer')
    #je trace l'indicateur
    ech.index=ech['Country']
    del ech['Country']
    x=np.array(ech.columns,dtype=int)
    fig.add_subplot(3,4,i+1)
    style=['-','--','-.',':']
    j=0
    for (country,y) in ech.sort_index().iterrows():
        plt.plot(x, np.array(y), label=country,linestyle=style[j%4])
        j+=1
    plt.title(indicateur)
    i+=1

    
    
    

In [ ]:
indicateurs=newdata.columns.tolist()[5:]

In [ ]:
newdata.columns=['Code','Country','Pop','Internet','Computer','sec%pop','ter%pop','proj_MYS','proj_popendsec_1000','ratio_sec','second_priv','inbound','ratio_ter','ter_per100k','exp_sec','exp_ter']

In [ ]:
newdata.head()

In [ ]:
newdata.head()

In [ ]:
L=newdata.columns.tolist()
L

In [ ]:
newdata=newdata[['Country','Population','TargetPop_internet','TargetPop_computer',\
                          'Barro-Lee: Percentage of population age 15+ with secondary s',\
                          'Barro-Lee: Percentage of population age 15+ with tertiary sc',\
                          'Wittgenstein Projection: Population in thousands by highest ',\
                          'Pupil-teacher ratio in secondary education (headcount basis)',\
                          'Enrolment in secondary education, private institutions, both',\
                          'Total inbound internationally mobile students, both sexes (n',\
                          'Pupil-teacher ratio in tertiary education (headcount basis)',\
                          'Enrolment in tertiary education per 100,000 inhabitants, bot']]
newdata.columns=['Country','Pop','Int','Comp','%pop_sec','%pop_ter','proj_postsec_1000','ratio_sec','private_sec','inbound','ratio_ter','ter_enrol_100000']

In [ ]:
newdata.head()

In [ ]:
newdata.isnull().sum()

In [ ]:
#un seul nan pour l'australie sur le ratio prof eleves dans le seocndaire: on le remplie avec une valeur de l'ordre de UK,US,Canada:16
newdata.fillna(16,inplace=True)
newdata.head(1)

In [ ]:
#on va transformer les %pop en pop car ce sont les nombres de clients qui nous intéressent.
#de meme avec les proj de fin de secondaire pour 1000 habitants et les enrollment dans le supérieur pour 100 000
#Pour avoir un nombre d'étudiants total on va rajouter dans ce dernier les inbound students
newdata['secondarylevelpop']=newdata['Pop']*newdata['%pop_sec']/100
newdata['superieurlevelpop']=newdata['Pop']*newdata['%pop_ter']/100
newdata['secondarylevelproj']=newdata['Pop']*newdata['proj_postsec_1000']/1000
newdata['students']=newdata['Pop']*newdata['ter_enrol_100000']/100000+newdata['inbound']
newdata=newdata[['Country','Int','Comp','ratio_sec','ratio_ter','private_sec','secondarylevelpop',\
                 'superieurlevelpop','secondarylevelproj','students']]


In [ ]:
newdata=newdata[['Country','Int','Comp','ratio_sec','ratio_ter','private_sec','secondarylevelpop','superieurlevelpop','secondarylevelproj','students']]

In [ ]:
newdata.head()

In [ ]:
#En dessous des tests que j'avais fait pour tracer les graphiques des divers indicateurs que j'ai ensuite fait avec une boucle plus haut.

In [ ]:
score=newdata.copy()
score['techno']=score['Int']+score['Comp']
score['now']=score['secondarylevelpop']+score['superieurlevelpop']+\
             score['students']*score['ratio_ter']+score['private_sec']*score['ratio_sec']
score['future']=score['secondarylevelproj']*(score['ratio_ter']+score['ratio_sec'])
scorefinal=score[['Country','techno','now','future']].copy()

q_tech=[scorefinal.techno.quantile(0.25*i) for i in range(1,4)]
q_act=[scorefinal.now.quantile(0.25*i) for i in range(1,4)]
q_futur=[scorefinal.future.quantile(0.25*i) for i in range(1,4)]

def quartile(x,L):
    if x<L[0]:
        return 0
    elif x<L[1]:
        return 1
    elif x<L[2]:
        return 2
    else:
        return 3
    
scorefinal.techno=[quartile(t,q_tech) for t in scorefinal.techno]
scorefinal.now=[quartile(t,q_act) for t in scorefinal.now]
scorefinal.future=[quartile(t,q_futur) for t in scorefinal.future]

scorefinal['Total']=scorefinal.techno+scorefinal.now+scorefinal.future
scorefinal.sort_values(by='Total',ascending=False,inplace=True)
scorefinal=scorefinal.iloc[:8]

In [ ]:
selection=scorefinal['Country'].tolist()

In [ ]:
echantillonfinal=echantillon[echantillon['Country Name'].isin(selection)]

In [ ]:
codes=['UIS.TE_100000.56','PRJ.POP.ALL.4.MF']
fig = plt.figure(figsize=(14,8))
k=1
for i in codes:
    ech=echantillonfinal[echantillonfinal['Indicator Code']==i].copy()
    indicateur=ech.iloc[0]['Indicator Name']
    ech.dropna(axis=1,how='all',inplace=True)
    del ech['Country Code']
    del ech['Indicator Name']
    del ech['Indicator Code']
    #Je déplace la colonne country en fin de tableau pour faire un ffill
    ech['Country']=ech['Country Name']
    del ech['Country Name']
    ech=ech.ffill(axis=1)
    ech2=ech[[ech.columns.tolist()[-1],ech.columns.tolist()[-2]]]
    ech2.columns=['Country',indicateur]
    newdata=pd.merge(newdata,ech2,on='Country',how='outer')
    #je trace l'indicateur
    ech.index=ech['Country']
    del ech['Country']
    x=np.array(ech.columns,dtype=int)
    fig.add_subplot(2,1,k)
    style=['-','--','-.',':']
    couleur=['blue','red']
    j=0
    for (country,y) in ech.sort_index().iterrows():
        plt.plot(x, np.array(y), label=country,linestyle=style[j%4],color=couleur[j//4])
        j+=1
    plt.legend(loc='upper left')
    plt.title(indicateur)
    
    k+=1


In [ ]:
fig = plt.figure(figsize=(21,14))
ax = plt.axes()

for (country,y) in tableauexpsec.iterrows():
   plt.plot(x, np.array(y), label=country)


# Les labels
plt.title("Un exemple de graphe")

# La légende est générée à partir de l'argument label de la fonctio
# plot. L'argument loc spécifie le placement de la légende
plt.legend(loc='upper left');

# Titres des axes
ax = ax.set(xlabel='x', ylabel='% budget alloué au secondaire')


In [ ]:
df=expendituressec[['2014','Country']]
df.columns=['Dépenses_gvt_secondaire','Country']
payscibles=pd.merge(payscibles,df,on='Country',how='outer')

In [ ]:
payscibles

In [ ]:
#Pas grand chose à en tirer... sauf éventuellement un classement en 3 groupes...

In [ ]:
expenditurester=newdata[newdata['Indicator Code']=='UIS.XGOVEXP.IMF.56'].copy()
expenditurester.dropna(axis=1,how='all',inplace=True)
del expenditurester['Country Code']
del expenditurester['Indicator Name']
del expenditurester['Indicator Code']
expenditurester

In [ ]:
expenditurester.index=expenditurester['Country Name']
tableaudepter=expenditurester.copy()
tableaudepter['Country']=tableaudepter['Country Name']
del tableaudepter['Country Name']
tableaudepter=tableaudepter.ffill(axis=1)
tabledepter=tableaudepter.copy()
depenses_sup=tableaudepter[['Country','2014']]
del tabledepter['Country']
x=np.array(tabledepter.columns,dtype=int)
fig = plt.figure(figsize=(21,14))
ax = plt.axes()

for (country,y) in tabledepter.iterrows():
   plt.plot(x, np.array(y), label=country)


# Les labels
plt.title("Dépenses supérieur")

# La légende est générée à partir de l'argument label de la fonctio
# plot. L'argument loc spécifie le placement de la légende
plt.legend(loc='upper left');

# Titres des axes
ax = ax.set(xlabel='x', ylabel='% budget alloué au tertiaire')


In [ ]:
depenses_sup.columns=['Country','Depenses_superieur']
payscibles

In [ ]:
payscibles=pd.merge(payscibles,depenses_sup,on='Country',how='outer')

In [ ]:
#Regardons les % de la population ayant été secondaires ou dans le supérieur  
peoplesec=newdata[newdata['Indicator Code']=='BAR.SEC.ICMP.15UP.ZS'].copy()
peoplesec.dropna(axis=1,how='all',inplace=True)
peoplesec.index=peoplesec['Country Name']
pourcentage_sec=peoplesec[['Country Name','2010']]
del peoplesec['Country Code']
del peoplesec['Indicator Name']
del peoplesec['Indicator Code']
peoplesec.index=peoplesec['Country Name']
pourecentage_sec=peoplesec[['Country Name','2010']]
tablepopsec=peoplesec.copy()
del tablepopsec['Country Name']
x=np.array(tablepopsec.columns,dtype=int)
fig = plt.figure(figsize=(21,14))
ax = plt.axes()

for (country,y) in tablepopsec.iterrows():
   plt.plot(x, np.array(y), label=country)


# Les labels
plt.title("15+ dans le secondaire")

# La légende est générée à partir de l'argument label de la fonctio
# plot. L'argument loc spécifie le placement de la légende
plt.legend(loc='upper left');

# Titres des axes
ax = ax.set(xlabel='x', ylabel='% population dans le secondaire')

In [ ]:
pourcentage_sec.columns=['Country','Acces_secondaires_%']
payscibles=pd.merge(payscibles,pourcentage_sec,on='Country',how='outer')

In [ ]:
#Regardons les % de la population ayant été secondaires ou dans le supérieur  
peopleter=newdata[newdata['Indicator Code']=='BAR.TER.ICMP.15UP.ZS'].copy()
peopleter.dropna(axis=1,how='all',inplace=True)
del peopleter['Country Code']
del peopleter['Indicator Name']
del peopleter['Indicator Code']
peopleter.index=peopleter['Country Name']
acces_ter=peopleter[['Country Name','2010']]
tablepopter=peopleter.copy()
del tablepopter['Country Name']
x=np.array(tablepopter.columns,dtype=int)
fig = plt.figure(figsize=(21,14))
ax = plt.axes()

for (country,y) in tablepopter.iterrows():
   plt.plot(x, np.array(y), label=country)


# Les labels
plt.title("15+ dans le supérieur")

# La légende est générée à partir de l'argument label de la fonctio
# plot. L'argument loc spécifie le placement de la légende
plt.legend(loc='upper left');

# Titres des axes
ax = ax.set(xlabel='x', ylabel='% population accés au supérieur')

In [ ]:
acces_ter.columns=['Country','Acces_superieur_%']
payscibles=pd.merge(payscibles,acces_ter)
payscibles

In [ ]:
#Les projections
#Années école 20-39
#Regardons le nombre d'année moyen à l'école  
school2039=newdata[newdata['Indicator Code']=='PRJ.MYS.2039.MF'].copy()
school2039.dropna(axis=1,how='all',inplace=True)
del school2039['Country Code']
del school2039['Indicator Name']
del school2039['Indicator Code']
school2039.index=school2039['Country Name']
table2039=school2039.copy()
del table2039['Country Name']
x=np.array(table2039.columns,dtype=int)
fig = plt.figure(figsize=(21,14))
ax = plt.axes()

for (country,y) in table2039.iterrows():
   plt.plot(x, np.array(y), label=country)


# Les labels
plt.title("Nb d'années moyen en formation")

# La légende est générée à partir de l'argument label de la fonctio
# plot. L'argument loc spécifie le placement de la légende
plt.legend(loc='upper left');

# Titres des axes
ax = ax.set(xlabel='x', ylabel='années')


In [ ]:
#Pas très pertinent au final

In [ ]:
#Pas sur que cela apporte grand chose, témoigne d'une potentiel croissant de personnes dans le secondaires et le supérieur quel que soient les pays.

In [ ]:
#Regardons les 15+ ayant atteint la fin du secondaire 
sec15=newdata[newdata['Indicator Code']=='PRJ.POP.ALL.4.MF'].copy()
sec15.dropna(axis=1,how='all',inplace=True)
del sec15['Country Code']
del sec15['Indicator Name']
del sec15['Indicator Code']
sec15.index=sec15['Country Name']
tablesec15=sec15.copy()
people_endsec_2030=sec15[['Country Name','2030']]
del tablesec15['Country Name']
x=np.array(tablesec15.columns,dtype=int)
fig = plt.figure(figsize=(21,14))
ax = plt.axes()

for (country,y) in tablesec15.iterrows():
   plt.plot(x, np.array(y), label=country)


# Les labels
plt.title("15 ans et plus ayant au moins atteint la fin du secondaire")

# La légende est générée à partir de l'argument label de la fonctio
# plot. L'argument loc spécifie le placement de la légende
plt.legend(loc='upper left');

# Titres des axes
ax = ax.set(xlabel='x', ylabel='milliers')

In [ ]:
people_endsec_2030.columns=['Country','People_Number_End_Seconday_2030']
payscibles=pd.merge(payscibles,people_endsec_2030,on='Country',how='outer')

In [ ]:
#La on a quand meme 5 pays qui se détachent nettement dans les 20-30 ans à venir
# Inde, Chine, USA, Japon et Philippines

In [ ]:
newdata.groupby(['Indicator Code','Indicator Name']).count()

In [ ]:
#Voyons le nombre d'élèves dans le privé dans le secondaire: 'UIS.E.23.PR'
prive=newdata[newdata['Indicator Code']=='UIS.E.23.PR'].copy()
prive.dropna(axis=1,how='all',inplace=True)
del prive['Country Code']
del prive['Indicator Name']
del prive['Indicator Code']
prive.index=prive['Country Name']
tableprive=prive.copy()
tableprive['Country']=tableprive['Country Name']
tableprive=tableprive.ffill(axis=1)
student_private=tableprive[['Country','2014']]
student_private.columns=['Country','Students_in_privateinstitutions_2014']
student_private


In [ ]:
payscibles=pd.merge(payscibles,student_private)

In [ ]:
tableprive

In [ ]:
#Rations profs élèves:
#Secondaire:

ptsec=newdata[newdata['Indicator Code']=='SE.SEC.ENRL.TC.ZS'].copy()
ptsec.dropna(axis=1,how='all',inplace=True)
del ptsec['Country Code']
del ptsec['Indicator Name']
del ptsec['Indicator Code']
ptsec.index=ptsec['Country Name']

tableptsec=ptsec.copy()
tableptsec['Country']=tableptsec['Country Name']
del tableptsec['Country Name']
tableptsec=tableptsec.ffill(axis=1)
ratiosec=tableptsec[['Country','2015']]
ratiosec.columns=['Country','Eleves_par_prof_Secondaire_2015']
del tableptsec['Country']

x=np.array(tableptsec.columns,dtype=int)
fig = plt.figure(figsize=(21,14))
ax = plt.axes()

for (country,y) in tableptsec.iterrows():
   plt.plot(x, np.array(y), label=country)


# Les labels
plt.title("Ration prof élève secondaire")

# La légende est générée à partir de l'argument label de la fonctio
# plot. L'argument loc spécifie le placement de la légende
plt.legend(loc='upper left');

# Titres des axes
ax = ax.set(xlabel='x', ylabel='Nb élève par prof')



In [ ]:
payscibles=pd.merge(payscibles,ratiosec,on='Country',how='outer')
payscibles.shape

In [ ]:
#Et dans le supérieur:
ptter=newdata[newdata['Indicator Code']=='UIS.PTRHC.56'].copy()
ptter.dropna(axis=1,how='all',inplace=True)
del ptter['Country Code']
del ptter['Indicator Name']
del ptter['Indicator Code']
ptter.index=ptter['Country Name']

tableptter=ptter.copy()
tableptter['Country']=tableptter['Country Name']
del tableptter['Country Name']
tableptter=tableptter.ffill(axis=1)
ratioter=tableptter[['Country','2015']]
ratioter.columns=['Country','Eleves_par_prof_Superieur_2015']
del tableptter['Country']


x=np.array(tableptter.columns,dtype=int)
fig = plt.figure(figsize=(21,14))
ax = plt.axes()

for (country,y) in tableptter.iterrows():
   plt.plot(x, np.array(y), label=country)


# Les labels
plt.title("Ratio prof élève supérieur")

# La légende est générée à partir de l'argument label de la fonctio
# plot. L'argument loc spécifie le placement de la légende
plt.legend(loc='upper left');

# Titres des axes
ax = ax.set(xlabel='x', ylabel='Nb élève par prof')


In [ ]:

payscibles=pd.merge(payscibles,ratioter,on='Country',how='outer')

In [ ]:
#nombre de personnes dans le supérieur pour 100 000
#UIS.TE_100000.56
ter=newdata[newdata['Indicator Code']=='UIS.TE_100000.56'].copy()
ter.dropna(axis=1,how='all',inplace=True)
del ter['Country Code']
del ter['Indicator Name']
del ter['Indicator Code']
ter.index=ptter['Country Name']
students=ter[['Country Name','2014']]
students.columns=['Country','NbEtudiants_100000hab']


tableter=ter.copy()
tableter['Country']=tableter['Country Name']
del tableter['Country Name']
tableter=tableter.ffill(axis=1)

students=tableter[['Country','2014']]
students.columns=['Country','NbEtudiants_100000hab']

payscibles=pd.merge(payscibles,students,on='Country',how='outer')

In [ ]:
#Et on fini avec le nombre d'étudiants étranger dans le supérieur
#UIS.TE_100000.56
etr=newdata[newdata['Indicator Code']=='UIS.MS.56.T'].copy()
etr.dropna(axis=1,how='all',inplace=True)
del etr['Country Code']
del etr['Indicator Name']
del etr['Indicator Code']
etr.index=etr['Country Name']

tableetr=etr.copy()
del tableetr['Country Name']
tableetr.ffill(axis=1,inplace=True)


tableetr=etr.copy()
tableetr['Country']=tableetr['Country Name']
del tableetr['Country Name']
tableetr=tableetr.ffill(axis=1)

studentsetr=tableetr[['Country','2014']]
studentsetr.columns=['Country','NbEtudiants_etranger']

payscibles=pd.merge(payscibles,studentsetr,on='Country',how='outer')




In [ ]:
payscibles.columns=['Country Code', 'Country', 'TargetPop_internet', 'TargetPop_computer','Dépenses_gvt_secondaire', 'Depenses_superieur', 'Acces_secondaires_%','Acces_superieur_%', 'People_Number_End_Seconday_2030','Students_in_privateinstitutions_2014','Eleves_par_prof_Secondaire_2015', 'Eleves_par_prof_Superieur_2015','NbEtudiants_100000hab_2014','NbEtudiants_etranger_2014']

In [ ]:
population=internet[['Country','Population']]
payscibles=pd.merge(payscibles,population)

In [ ]:
payscibles.to_csv('preselection.csv')